In [69]:
import pandas as pd
from konlpy.tag import Hannanum
import yfinance as yf
import datetime
import numpy as np

In [61]:
tmp = df_process(df)

In [62]:
tmp

,제목
일자,
2022-03-16,"신세계百, 강남점 센텀시티점에 국내 최초 '베이비 디올' 매장 오픈"
2022-03-16,"이동근 ""중대재해법, 기업 부담 커 윤석열 정부, 개선해달라"""
2022-03-16,'위기에도 혁신' K-배터리 한자리 반도체 신화 넘본다
2022-03-16,[솔선守法] 중대재해법 첫 사건
2022-03-16,"코스피, 기관 매수에 상승 마감 하루만에 '7만전자' 회복"
...,...
2022-03-31,"전세대출도 5% 넘었다 한은, 금리인상 스텝 한템포 쉬어가나"
2022-03-31,"가상자산, 성능과 안정의 시대로 테라 솔라나 급부상"
2022-03-31,“원전 생태계 복원 필요성 에너지 자립도 높여야”


In [64]:
#for i in tmp.index.unique():
    

In [65]:
start_date_str = datetime.strptime(tmp.index.unique()[0], '%Y-%m-%d')
end_date_str = datetime.strptime(tmp.index.unique()[-1], '%Y-%m-%d')

AttributeError: module 'datetime' has no attribute 'strptime'

In [66]:
str(tmp.index.unique()[0])[0:10]

'2022-03-16'

In [70]:
start_date_str = str(tmp.index.unique()[0])[0:10]
end_date_str = str(tmp.index.unique()[-1])[0:10]

check = yf.download('^KS11', start_date_str, end_date_str, progress=False)
check['signal'] = np.where((check['Close'] - check['Open'])> 0 ,1, 0)

In [71]:
check

,Open,High,Low,Close,Adj Close,Volume,signal
Date,,,,,,,
2022-03-16,2649.850098,2659.899902,2630.030029,2659.229980,2659.229980,736000,1
2022-03-17,2703.270020,2714.000000,2694.510010,2694.510010,2694.510010,545500,0
2022-03-18,2695.639893,2708.159912,2688.709961,2707.020020,2707.020020,643500,1
2022-03-21,2717.810059,2717.810059,2683.689941,2686.050049,2686.050049,502000,0
2022-03-22,2686.679932,2712.139893,2686.580078,2710.000000,2710.000000,489100,1
2022-03-23,2727.120117,2742.270020,2720.830078,2735.050049,2735.050049,883000,1
2022-03-24,2716.250000,2729.659912,2705.139893,2729.659912,2729.659912,959600,1
2022-03-25,2740.830078,2740.949951,2722.360107,2729.979980,2729.979980,599800,0
2022-03-28,2721.219971,2737.459961,2707.739990,2729.560059,2729.560059,768000,1


In [84]:
check.index[0]

Timestamp('2022-03-16 00:00:00')

In [85]:
tmp.index[0]

Timestamp('2022-03-16 00:00:00')

In [87]:
check.loc[check.index[0], 'signal']

1

In [86]:
tmp[tmp.index == check.index[0]]['signal'] = check.loc[check.index[0], 'signal']

/var/folders/0k/zyr795gx3hl6x1nytrrntcnc0000gn/T/ipykernel_10121/3913218476.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[tmp.index == check.index[0]]['signal'] = check.loc[check.index[0], 'signal']


In [88]:
tmp

,제목
일자,
2022-03-16,"신세계百, 강남점 센텀시티점에 국내 최초 '베이비 디올' 매장 오픈"
2022-03-16,"이동근 ""중대재해법, 기업 부담 커 윤석열 정부, 개선해달라"""
2022-03-16,'위기에도 혁신' K-배터리 한자리 반도체 신화 넘본다
2022-03-16,[솔선守法] 중대재해법 첫 사건
2022-03-16,"코스피, 기관 매수에 상승 마감 하루만에 '7만전자' 회복"
...,...
2022-03-31,"전세대출도 5% 넘었다 한은, 금리인상 스텝 한템포 쉬어가나"
2022-03-31,"가상자산, 성능과 안정의 시대로 테라 솔라나 급부상"
2022-03-31,“원전 생태계 복원 필요성 에너지 자립도 높여야”


In [59]:
def df_process (df):
    
    df['일자'] = pd.to_datetime(df['일자'], format='%Y%m%d')
    df = df.set_index('일자')

    df = df.sort_index(ascending=True)    

    return df[['제목']]

In [9]:
df = pd.read_excel('NewsResult_20220315-20220331.xlsx')

/opt/homebrew/Caskroom/miniforge/base/envs/NLP/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [60]:
df.head()

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,2.100601e+06,2022-03-31,한국경제,윤아영,'IFC인수전' 미래에셋과 손잡는 유통업체는 어디,경제>산업_기업,경제>무역,경제>금융_재테크,NaN,NaN,NaN,NaN,"IFC인수전,여의도,IFC,강북","파크원,미래에셋,미래에셋자산운용,IFC,더현대,미래에셋컨소시엄,신세계프라퍼티,이지스...","IFC인수전,미래에셋,유통업체,기사,08:27,마켓인사이트,게재,기사,다음달,입찰,...","ifc,여의도,투자자,유통업체,쇼핑몰,관계자,미래에셋자산운용,더현대,신세계프라퍼티,...",이 기사는 03월 31일 08:27 “마켓인사이트”에 게재된 기사입니다. \n \n...,https://www.hankyung.com/economy/article/20220...,NaN
1,2.100601e+06,2022-03-31,한국경제,윤아영,"카사, 네번째 상장 건물로 동대문 부티크호텔 공개",경제>증권_증시,지역>경남,경제>부동산,NaN,NaN,NaN,예창완,"동대문역사문화공원역,동대문,부티크호텔,2·4·5호선,부티크","카사,플라자,Kasa,케이엔에이치,마켓인사이트,르릿","카사,건물,번째,상장,공개,동대문,부티크호텔,기사,09:18,마켓인사이트,게재,기사...","카사,부티크호텔,동대문,부동산,르릿,댑스,마켓인사이트,임대차,동대문역사문화공원역,예...",이 기사는 03월 31일 09:18 “마켓인사이트”에 게재된 기사입니다. \n \n...,https://www.hankyung.com/economy/article/20220...,NaN
2,2.100601e+06,2022-03-31,한국경제,윤아영,"SK D&D-국민연금, 리츠 통해 남대문 일대 고층 오피스 개발",경제>부동산,지역>강원,지역>경기,NaN,NaN,NaN,NaN,"중구,서울,봉래구역,봉래,남대문로,서울역,남대문,남대문로5가","국민연금공단,국민연,DDVIC1호,SK,국토교통부,마켓인사이트,디앤디인베스트먼트","SK,D&D,국민연금,리츠,남대문,일대,고층,오피스,기사,06:07,마켓인사이트,게...","ddivc1호리츠,봉래,남대문로,남대문,sk,서울,디앤디인베스트먼트,디디아이브이,중...",이 기사는 03월 31일 06:07 “마켓인사이트”에 게재된 기사입니다. \n \n...,https://www.hankyung.com/economy/article/20220...,NaN
3,2.100601e+06,2022-03-31,한국경제,이태호,신평 3사 모두 현대두산인프라 신용등급 상향,경제>금융_재테크,경제>자동차,경제>반도체,NaN,NaN,NaN,김봉환,신평,"현대두산인프라코어,현대중공업,현대두산인프라,유상증자,한국기업평가,한국신용평가,마켓인...","신평,상향,현대두산인프라,신용,등급,기사,18:21,마켓인사이트,게재,기사,나이스신...","현대두산인프라코어,나신평,신용등급,현대두산인프라,상각전,김봉환,마켓인사이트,6833...",이 기사는 03월 31일 18:21 “마켓인사이트”에 게재된 기사입니다. \n \n...,https://www.hankyung.com/finance/article/20220...,NaN
4,2.100601e+06,2022-03-31,한국경제,김진수,[데스크 칼럼] 국토교통부 '홀대론',정치>청와대,경제>부동산,경제>산업_기업,NaN,NaN,NaN,"노형욱,박근혜,문재인,윤,유일호,윤석열,강호일,김현미,변창흠,서승환","서울,강남,신성,하남,남양주,왕숙,서울시,경기도,수도권","인수위,하마평,기재부,산업부,국토부,서울시,LH(,정부,한국토지주택공사,청와대,기용...","국토,교통부,홀대,대통령,인수,위원회,출범,인수위,당선인,윤석열,대통령,공약,토대,...","국토부,부동산,인수위,인수위원,당선인,서울시,윤석열,산업부,재건축,경기도,서승환,박근혜",제20대 대통령직인수위원회가 출범한 지 2주가 지났다. 인수위는 윤석열 대통령 당선...,https://www.hankyung.com/opinion/article/20220...,NaN


In [11]:
df_process(df)

,제목
일자,
2022-03-31,'IFC인수전' 미래에셋과 손잡는 유통업체는 어디
2022-03-31,"카사, 네번째 상장 건물로 동대문 부티크호텔 공개"
2022-03-31,"SK D&D-국민연금, 리츠 통해 남대문 일대 고층 오피스 개발"
2022-03-31,신평 3사 모두 현대두산인프라 신용등급 상향
2022-03-31,[데스크 칼럼] 국토교통부 '홀대론'
...,...
2022-03-16,"홍남기 ""2월 고용시장, 모든 면에서 개선세 뚜렷"""
2022-03-16,"“레이, 中 치과 시장 진출로 성장할 것”"
2022-03-16,유재영 SK이노베이션 울산CLX 총괄 “親지구적 그린플랜트로 진화”


In [12]:
okt = Hannanum()

In [19]:
from konlpy.tag import Mecab

'-d /usr/local/lib/mecab/dic/mecab-ko-dic'

In [20]:
mcb = Mecab()
text = "안녕하세요 제 이름은 이선우 입니다."


In [21]:
mcb.pos(text, flatten=False, join=True)

[['안녕/NNG', '하/XSV', '세요/EP+EF'],
 ['제/NP'],
 ['이름/NNG', '은/JX'],
 ['이선우/NNP'],
 ['입니다/VCP+EF', './SF']]

In [22]:
mcb.morphs(text)

['안녕', '하', '세요', '제', '이름', '은', '이선우', '입니다', '.']

In [15]:
ret

NameError: name 'ret' is not defined

In [ ]:
!pip install eunjeon

  Using cached eunjeon-0.4.0.tar.gz (34.7 MB)
  Created wheel for eunjeon: filename=eunjeon-0.4.0-cp38-cp38-macosx_11_0_arm64.whl size=34077890 sha256=59a34a966b11252002d060f27a87a6815908d3f7613ecbb6fece3252237a65cb
  Stored in directory: /Users/Sun/Library/Caches/pip/wheels/4b/1d/e9/bfa4c18245fc292aaa650fe6dd982a4feddfaab49f3b8f9f25
Successfully built eunjeon


In [ ]:
from eunjeon import MeCab

m = MeCab()

m.pos("안녕하세요 제 이름은 이선우 입니다.")

ImportError: cannot import name 'MeCab' from 'eunjeon' (/opt/homebrew/Caskroom/miniforge/base/envs/NLP/lib/python3.8/site-packages/eunjeon/__init__.py)